In [ ]:
# Day 9 - Part 1:
# Given a list of 2D points
# find the two points, that create the largest rectangular area when placed at opposite corners.


def solve_part1(file_path):
    # read in points from data
    points = []
    with open(file_path, "r") as f:
        points = [tuple(map(int, line.strip().split(","))) for line in f.readlines()]
    # calc area of each pair, keep track of max area
    pair = None
    max_area = 0
    for i, p1 in enumerate(points):
        for j, p2 in enumerate(points[i + 1 :]):
            # calculate area - remember, points are inclusive
            area = (abs(p1[0] - p2[0]) + 1) * (abs(p1[1] - p2[1]) + 1)
            if area > max_area:
                max_area = area
                pair = (p1, p2)
    return max_area


print(f"Part 1 - Test Data: {solve_part1('./data/day9-test.txt')}")
print(f"Part 1 - Full Data: {solve_part1('./data/day9-data.txt')}")

In [ ]:
# Day 9 - Part 2:
# Each point (red-tiles) is connected to the next point (green-tiles) in the list.
# the list wraps and the last point connects to the first point.
# when a loop is formed that connects all point, all interior points are filled (green-tiles).
# Find the largest rectangle formed by red tiles, that is completely filled with red or green tiles.


def solve_part2(file_path):
    # read in points from data
    points = []
    with open(file_path, "r") as f:
        points = tuple(
            tuple(map(int, line.strip().split(","))) for line in f.readlines()
        )

    # gen pairwise combinations of points, calc area to set priority
    pairs = []
    max_area = 0
    for i, p1 in enumerate(points):
        for j, p2 in enumerate(points[i + 1 :]):
            # calculate area - remember, points are inclusive
            area = (abs(p1[0] - p2[0]) + 1) * (abs(p1[1] - p2[1]) + 1)
            # if area > max_area and rect_in_polygon(p1, p2, points):
            #     max_area = area
            #     pair = (p1, p2)
            pairs.append((p1, p2, area))

    # sort pairs by area, descending
    pairs.sort(key=lambda x: x[2], reverse=True)
    # check each pair if rectangle is in polygon
    for p1, p2, area in pairs:
        if area == 24:
            print(f"Checking pair: {p1}, {p2} with area {area}")
        if rect_in_polygon(p1, p2, points):
            max_area = area
            break
    return max_area


def point_in_polygon(point, polygon):
    x, y = point
    n = len(polygon)
    # count intersections of a ray from point to the right with polygon edges
    intersections = 0
    p1x, p1y = polygon[0]
    for i in range(1, n + 1):
        # is point a corner?
        if (x, y) == (p1x, p1y):
            return True
        # get end-point of edge
        p2x, p2y = polygon[i % n]
        # compute the x coordinate of the intersection of the edge with the horizontal line at y
        xinters = -float("inf")
        # avoid division by zero if p1y equals p2y
        # and check for horizontal edge case
        if p1y != p2y:
            xinters = p1x + (p2x - p1x) / (p2y - p1y) * (y - p1y)
        elif y == p1y and (x > p1x) != (x > p2x):
            return True  # point is on horizontal edge

        # check if point is within the y bounds and left or equal to edge in x-direction
        if (y > p1y) != (y > p2y) and x <= xinters:
            # return True if point is on edge
            if x == xinters:
                return True
            # count intersection if point is to the left of intersection
            intersections += 1
        # prepare for beginning of next edge
        p1x, p1y = p2x, p2y
    return intersections % 2 == 1


# function to check if rectangle is inside polygon
def rect_in_polygon(c1, c2, polygon):
    (x1, y1), (x2, y2) = c1, c2
    if x1 > x2:
        x1, x2 = x2, x1
    if y1 > y2:
        y1, y2 = y2, y1
    # check corners first
    corners = [(x1, y1), (x1, y2), (x2, y1), (x2, y2)]
    for corner in corners:
        if not point_in_polygon(corner, polygon):
            return False
    # check for segment intersections with polygon edges
    p1 = polygon[0]
    # check each edge of rectangle
    rect_edges = [
        ((x1, y1), (x1, y2)),
        ((x1, y2), (x2, y2)),
        ((x2, y2), (x2, y1)),
        ((x2, y1), (x1, y1)),
    ]
    # check each edge of polygon to see if it intersects with any edge of rectangle
    for i in range(1, len(polygon) + 1):
        p2 = polygon[i % len(polygon)]
        for r1, r2 in rect_edges:
            if segments_intersect(p1, p2, r1, r2):
                return False
        p1 = p2
    return True


def segments_intersect(p1, p2, q1, q2):
    # How this works:
    # We use the cross product to determine orientation of triplets
    # and check if the segments straddle each other.

    # General case:
    # (p1, p2, q1) and (p1, p2, q2) have different orientations ( q1 and q2 are on different sides of line p1 p2)
    # (q1, q2, p1) and (q1, q2, p2) have different orientations ( p1 and p2 are on different sides of line q1 q2)

    # In our discrete case, if p1 or p2 equals q1 or q2, we do not consider that an intersection.
    if p1 == q1 or p1 == q2 or p2 == q1 or p2 == q2:
        return False

    def orientation(a, b, c):
        # cross product of vector ab and bc
        val = (b[0] - a[0]) * (c[1] - b[1]) - (b[1] - a[1]) * (c[0] - b[0])
        if val == 0:
            return 0  # collinear
        return 1 if val < 0 else 2  # clockwise or counter clockwise

    o1 = orientation(p1, p2, q1)
    o2 = orientation(p1, p2, q2)
    o3 = orientation(q1, q2, p1)
    o4 = orientation(q1, q2, p2)

    # Special discrete case for our problem:
    # if any orientation is collinear (e.i. an end of a segment rests on another segment), we do not consider that an intersection
    if o1 == 0 or o2 == 0 or o3 == 0 or o4 == 0:
        return False

    # General case
    if o1 != o2 and o3 != o4:
        return True

    # Special Cases - collinear points
    # COMMENTING OUT - do not consider collinear as intersecting in our problem
    # def on_segment(p, q, r):
    #     return (
    #         q[0] <= max(p[0], r[0])
    #         and q[0] >= min(p[0], r[0])
    #         and q[1] <= max(p[1], r[1])
    #         and q[1] >= min(p[1], r[1])
    #     )

    # Special Cases - collinear points
    # COMMENTING OUT - do not consider collinear as intersecting in our problem
    # # p1, p2 and q1 are collinear and q1 lies on segment p1p2
    # if o1 == 0 and on_segment(p1, q1, p2):
    #     return True
    # # p1, p2 and q2 are collinear and q2 lies on segment p1p2
    # if o2 == 0 and on_segment(p1, q2, p2):
    #     return True
    # # q1, q2 and p1 are collinear and p1 lies on segment q1q2
    # if o3 == 0 and on_segment(q1, p1, q2):
    #     return True
    # # q1, q2 and p2 are collinear and p2 lies on segment q1q2
    # if o4 == 0 and on_segment(q1, p2, q2):
    #     return True

    return False


print(f"Part 2 - Test Data: {solve_part2('./data/day9-test.txt')}")
print(f"Part 2 - Full Data: {solve_part2('./data/day9-data.txt')}")